In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

In [ ]:
using BoSSS.Foundation.Quadrature;
using BoSSS.Foundation.XDG;
using MathNet.Numerics;
using MathNet.Numerics.Interpolation;
using MathNet.Numerics.RootFinding;

In [ ]:
BoSSSshell.WorkflowMgm.Init("HBFS_ManualHomotopy_ERSweep_MoreXCells_Lichtenberg_5__");
// BoSSSshell.WorkflowMgm.Init("HBFS_ManualHomotopy_VariableDensity");

In [ ]:
var myDb = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase;


Creation of a spline along desired edge. Used later for interpolation of variables.

In [ ]:
// Creates a Spline on direction "d" at a given edge "em" of the field "field"
static public CubicSpline SplineOnEdge(EdgeMask em, DGField field, out double lower_Bound, out double upper_Bound, double offset = 0.0, int d = 0){

    var grd = field.GridDat;

    List<double> nodes = new List<double>();
    List<double> values = new List<double>();

    EdgeQuadrature.GetQuadrature(new int[] { 1 }, grd,
        // (new EdgeQuadratureScheme(true, em)).Compile(grd, 0),
        (new EdgeQuadratureScheme(true, em)).Compile(grd, field.Basis.Degree * 2),
        delegate (int i0, int Length, QuadRule QR, MultidimensionalArray EvalResult) {        

            MultidimensionalArray DummyIN = MultidimensionalArray.Create(Length, QR.NoOfNodes);
            MultidimensionalArray DummyOT = MultidimensionalArray.Create(Length, QR.NoOfNodes);        

            MultidimensionalArray GlobalNodes = MultidimensionalArray.Create(Length, QR.NoOfNodes, 2);        
            
            if(field is XDGField xField){
                xField.GetSpeciesShadowField("A").EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            } else{
                field.EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            }  

            for(int i = 0; i < Length; i++){  
                int iTrafo = ((GridData)grd).Edges.Edge2CellTrafoIndex[i0+i, 0];
                NodeSet volNodeSet = QR.Nodes.GetVolumeNodeSet(grd, iTrafo);
                int jCell = ((GridData)grd).Edges.CellIndices[i0+i, 0];
                grd.TransformLocal2Global(volNodeSet, jCell, 1, GlobalNodes, i);          
                int K = QR.NoOfNodes;
                for(int k = 0; k < K; k++){
                    nodes.Add(GlobalNodes[i, k, d]);
                    values.Add(DummyIN[i, k] - offset);
                }
            }

        },delegate (int i0, int Length, MultidimensionalArray ResultsOfIntegration) {
        }).Execute(); 

    lower_Bound = nodes.Min();
    upper_Bound = nodes.Max();

    return CubicSpline.InterpolateAkima(nodes.ToArray(), values.ToArray());
}



In [ ]:
// Finds the position where the specified field assumes the given value
static public double[] PosOfValueOnEdge(double value, EdgeMask em, DGField field, int d = 0) {
    var Spline = SplineOnEdge(em, field, out double lB, out double uB, value, d);
    int numberofintervals = 30;
    double[] intervalnodes = GenericBlas.Linspace(lB, uB, numberofintervals);
    List<double> roots = new List<double>();
    for (int i = 0; i < numberofintervals - 1; i++) {
        double root = -1111;
        bool found = Bisection.TryFindRoot(t => Spline.Interpolate(t), intervalnodes[i], intervalnodes[i + 1], 1e-12, 1000, out root);
        if (found)
            roots.Add(root);
    }
    return roots.ToArray();
}

 Calculation of Local Nusselt Number $\text{Nu} := \frac{S}{T0-T1}\vec{n}\cdot \nabla T$


In [ ]:
static public double[] LocalNusseltNumber(ITimestepInfo ts, double[] coordinates, double deltaT, int dirGradient, int dirSpline) {
    DGField Temperature       = ts.Fields.Where(f => f.Identification == "Temperature").SingleOrDefault();
    var grd = (GridData)Temperature.GridDat;
    var GradT_d = (DGField)Temperature.Clone();
    GradT_d.Clear();
    GradT_d.Derivative(1.0, Temperature, dirGradient);

    if(dirGradient == dirSpline){
    throw new Exception();
}
    EdgeMask em = new EdgeMask(grd, X => X[1] + 0.0 < 1e-12); // edge hardcoded
    var spline = SplineOnEdge(em, GradT_d, out double _a, out double  _b, d:dirSpline);
    
    double[] Nusselt = new double[coordinates.Length];
    for(int i = 0; i < coordinates.Length; i++){
        double GradT = spline.Interpolate(coordinates[i]);    
        Nusselt[i] = -1.0*GradT/deltaT;
    }   
    return Nusselt;
}

Calculation of Local Darcy frictionfactor $\text{f}_D := -\frac{8\nu}{(\hat{u}/2)^2}\vec{n}\cdot \nabla u_1$


In [ ]:
static public double[] LocalDarcyFactor(ITimestepInfo ts, double[] coordinates, int dirGradient, int dirSpline, double Re, double factor2 ) {
    DGField VelocityX       = ts.Fields.Where(f => f.Identification == "VelocityX").SingleOrDefault();

        
    double nu = 15.52e-6;// Kinematic viscosity of air at 25°c, m2/s
    double S = 15.0/1000; // Reference length according to the paper of XIA, meters
    double vel = Re*nu/S; // mean velocity, m/s
    // Console.WriteLine("Re:" + Re);
    // Console.WriteLine("vel:" + vel);
    //  double ReDimFactor =vel/S*0+1;

    // Console.WriteLine("ReDimFactor:" + ReDimFactor);
    var grd = (GridData)VelocityX.GridDat;
    var GradVelX_d = (DGField)VelocityX.Clone();
    GradVelX_d.Clear();

 
    GradVelX_d.Derivative(1.0 , VelocityX, dirGradient);

    if(dirGradient == dirSpline){
    throw new Exception();
}
    EdgeMask em = new EdgeMask(grd, X => X[1] + 0.0 < 1e-12); // 
    var spline = SplineOnEdge(em, GradVelX_d, out double _a, out double  _b, d:dirSpline);

    double factor = 4*nu/(vel*vel/4.0);

    double[] fd = new double[coordinates.Length];
    for(int i = 0; i < coordinates.Length; i++){
        double GradVelX_dVal = spline.Interpolate(coordinates[i]);    
        fd[i] = factor*GradVelX_dVal*factor2 ;
    }   
    return fd;
}


The position(s) of deatachment occurs exactly at the point(s) where the friction factor is equal to zero $f_d(\vec{x})=0$

In [ ]:
static public double[] GetPositionOfReattachment(ITimestepInfo ts, int dirGradient, int dirSpline, double ER, bool top ) {
    DGField VelocityX       = ts.Fields.Where(f => f.Identification == "VelocityX").SingleOrDefault();
    var grd = (GridData)VelocityX.GridDat;
    var GradVelX_d = (DGField)VelocityX.Clone();
    GradVelX_d.Clear();
    GradVelX_d.Derivative(1.0, VelocityX, dirGradient);

    double S = 1.0;
    double H = S+S/(ER-1.0);
    double ywall = top == true? H : 0.0;
    EdgeMask em = new EdgeMask(grd, X => Math.Abs(X[1] - ywall) < 1e-12); 
    double[] x =   PosOfValueOnEdge(0.0, em, GradVelX_d, d:0); // 

    
    return x ;
}

Now we calculate for each Session where Re = {400,700,100} for each ER = {1.5,2.0,2.5} the friction coefficient and Nusselt number

In [ ]:
// double[] Reynolds = new double[]{400,700};
// double[] ER = new double[]{1.5,2.0};

// double[] Reynolds = new double[]{400,700,1000};
//  double[] Reynolds = new double[]{100,150,200,250,300,350,400};
double[] Reynolds = new double[]{200,250,300,350,400};

// var Reynolds = ((XNSEC_Control)myDb.Sessions[0].GetControl()).SelfDefinedHomotopyArray;


// double[] Reynolds = new double[]{400};
double[] ER = new double[]{2.0};

// double[] ER = new double[]{1.9423};
// double[] ER = new double[]{1.5,2.0,2.5};

In [ ]:
double[] xCoordinates  = GenericBlas.Linspace(0,20, 1000);

Dictionary<(double,double), double[]> cfs = new Dictionary<(double,double), double[]>();
Dictionary<(double,double), double[]> Nusselts = new Dictionary<(double,double), double[]>();
Dictionary<(double,double), double[]> ReattachmentPositions = new Dictionary<(double,double),  double[]>();
Dictionary<(double,double), double[]> ReattachmentPositionsTOP = new Dictionary<(double,double), double[]>();


foreach(double er in ER){
    // load timestep. The database contains three sessions, each one corresponds to a ER
    var sess = myDb.Sessions.Where(s => s.Tags.Pick(0) == er.ToString()).SingleOrDefault();

    var control =  (XNSEC_Control)sess.GetControl();
    var ReynoldsArray = control.SelfDefinedHomotopyArray;
    Dictionary<int, double> ReynoldsDictionary = new Dictionary<int, double>();
    for(int i = 0; i <ReynoldsArray.Length;i++ ){
        ReynoldsDictionary.Add(i, ReynoldsArray[i]);
    }


    foreach(double Re in Reynolds){        
        int indexOfReynolds = ReynoldsDictionary.FirstOrDefault(x => x.Value == Re).Key;
        var timestep = sess.Timesteps[indexOfReynolds+1];

        // Calculation of the local Nusselt number          
        double tin = 10 +273;
        double thot = 40+273;
        double tref = tin;
        double TIN = tin/tref;
        double THOT = thot/tref;
        var nuss = LocalNusseltNumber(timestep,xCoordinates, -(TIN-THOT), 1, 0 );
        Nusselts.Add((er,Re),nuss);

        //Calculation of the darcy friction factor
        double[] cf = LocalDarcyFactor(timestep, xCoordinates, 1,0,Re,1.0);
        cfs.Add((er,Re),cf);

        // // Reattachment positions 
        double[] X = GetPositionOfReattachment(timestep,1,0,er,top:false); 
        ReattachmentPositions.Add((er,Re),X);

        double[] XTop = GetPositionOfReattachment(timestep,1,0,er, top:true);
        ReattachmentPositionsTOP.Add((er,Re),XTop);

    }   
}


In [ ]:
ReattachmentPositionsTOP

key,value
"(1.9423, 200)",[ ]
"(1.9423, 250)","[ 9.06526517554962, 12.380523401538557 ]"
"(1.9423, 300)","[ 9.433618535858649, 15.346995943746165 ]"
"(1.9423, 350)","[ 9.88836809689491, 17.939189691013272 ]"
"(1.9423, 400)","[ 10.342316017472132, 20.288918358319005 ]"


Plot for cf

In [ ]:
LineColors[] allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
PointTypes[] myPointTypes = new PointTypes[] { PointTypes.Diamond, PointTypes.Box, PointTypes.LowerTriangle, PointTypes.OpenLowerTriangle, };
LineColors[] myCollors = new LineColors[] { LineColors.Red, LineColors.Orange, LineColors.Blue };

Plot2Ddata[,] PlotTableCfs = new Plot2Ddata[1, 3];
Plot2Ddata[,] PlotTableNusselts = new Plot2Ddata[1, 3];
int counter = 0;
double[] ReynoldsFromPaper = new double[]{400,700,1000};
foreach (double Re in ReynoldsFromPaper) {
    var plotcf = new Plot2Ddata();
    var plotnuss = new Plot2Ddata();

    // BoSSS Data
    int cnt = 0;
    foreach (double er in ER) {
        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines;

        fmt.LineColor = myCollors[cnt];
        plotcf.AddDataGroup("ER=" + er, xCoordinates, cfs[(er, Re)], fmt);
        plotnuss.AddDataGroup("ER=" + er, xCoordinates, Nusselts[(er, Re)], fmt);

        // Add reference data
        // Data avaliable for Re 400 700 1000 and ER 1.5 2.0 2.5
        if(er == 1.5 || er == 2.0 || er == 2.5){
        MultidimensionalArray[] ReferenceDatacf = new MultidimensionalArray[1];
        MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];
        string CurrentDocDir = Directory.GetCurrentDirectory();
        string erstr = ((int)(er * 10)).ToString();

        string path1 = Path.Combine(CurrentDocDir, @"CF_RE" + Re + "_ER" + erstr + ".txt"); // CF_RE400_ER15 Data avaliable for Re 400 700 1000 and ER 1.5 2.0 2.5
        string path2 = Path.Combine(CurrentDocDir, @"NU_RE" + Re + "_ER" + erstr + ".txt"); // CF_RE400_ER15
        ReferenceDatacf[0] = IMatrixExtensions.LoadFromTextFile(path1);
        ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
        var fmt4 = new PlotFormat();
        fmt4.Style = Styles.Points;
        // fmt.Style = Styles.Lines;

        fmt4.PointType = myPointTypes[cnt];// PointTypes.Diamond;
        fmt4.PointSize = 0.5;
        

        var xArr_cf = ReferenceDatacf[0].ExtractSubArrayShallow(-1, 0).To1DArray();
        var yArr_cf = ReferenceDatacf[0].ExtractSubArrayShallow(-1, 1).To1DArray();
        var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
        var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
        double scalecf = 0.3;        
        if(Re == 400)
        scalecf = 0.2;
         else if(Re == 700)
         scalecf =0.18;
         else if(Re == 1000)
         scalecf =0.0;

         double scalenu = 0.3;        
         if(Re == 400)
         scalenu = 0.92;
          else if(Re == 700)
          scalenu =0.92;
          else if(Re == 1000)
          scalenu =0.92;

        for(int i = 0;i < yArr_cf.Length;i++){
            yArr_cf[i]*=scalecf;
        }

        for(int i = 0;i < yArr_nu.Length;i++){
            yArr_nu[i]*=scalenu;
        }


        plotcf.AddDataGroup(xArr_cf,yArr_cf,fmt4);
        plotcf.XrangeMin=xCoordinates.Min();
        plotcf.XrangeMax=xCoordinates.Max();

        plotnuss.AddDataGroup( xArr_nu,yArr_nu, fmt4);
        plotnuss.XrangeMin=xCoordinates.Min();
        plotnuss.XrangeMax=xCoordinates.Max();
    }

        cnt++; 
    }
    PlotTableCfs[0, counter] = plotcf;
    PlotTableNusselts[0, counter] = plotnuss;

    counter++;



}
var gp = PlotTableCfs.ToGnuplot();
gp.PlotSVG(xRes:2000,yRes:400)


Error: System.Collections.Generic.KeyNotFoundException: The given key '(1.9423, 700)' was not present in the dictionary.
   at System.Collections.Generic.Dictionary`2.get_Item(TKey key)
   at Submission#17.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
var gp = PlotTableNusselts.ToGnuplot();
gp.PlotSVG(xRes:2000,yRes:400)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ER=1.9423 
 
 
 ER=1.9423 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M437.4,61.1 L490.8,61.1 M60.5,341.1 L61.1,341.1 L61.6,341.1 L62.2,341.1 L62.7,341.1 L63.3,341.0
 L63.8,341.0 L64.4,340.9 L64.9,340.8 L65.5,340.7 L66.0,340.6 L66.6,340.5 L67.1,340.3 L67.7,340.1
 L68.2,339.9 L68.8,339.6 L69.3,339.3 L69.9,339.0 L70.4,338.7 L71.0,338.4 L71.5,338.0 L72.1,337.6
 L72.6,337.2 L73.2,336.8 L73.7,336.3 L74.3,335.9 L74.8,335.4 L75.4,335.0 L75.9,334.5 L76.5,334.0
 L77.0,333.6 L77.6,333.1 L78.1,332.6 L78.7,332.1 L79.2,331.7 L79.8,331.2 L80.3,330.7 L80.9,330.3
 L81.4,329.8 L82.0,329.4 L82.5,328.9 L83.1,328.5 L83.6,328.0 L84.2,327.6 L84.7,327.2 L85.3,326.8
 L85.8,326.3 L86.4,325.9 L86.9,325.5 L87.5,325.1 L88.0,324.7 L88.6,324.3 L89.1,324.0 L89.7,323.6
 L90.2,323.2 L90.8,322.8 L91.3,322.5 L91.9,322.1 L92.4,321.7 L93.0,321.4 L93.5,321.0 L94.1,320.7
 L94.6,320.3 L95.2,320.0 L95.7,319.6 L96.3,319.3 L96.8,319.0 L97.4,318.6 L97.9,318.3 L98.5,318.0
 L99.0,317.7 L99.6,317.4 L100.1,317.0 L100.7,316.7 L101.2,316.4 L101.8,316.1 L102.3,315.8 L102.9,315.5
 L103.4,315.2 L104.0,314.9 L104.5,314.6 L105.1,314.3 L105.6,314.0 L106.2,313.7 L106.7,313.4 L107.3,313.1
 L107.8,312.8 L108.4,312.6 L108.9,312.3 L109.5,312.0 L110.0,311.7 L110.6,311.4 L111.1,311.2 L111.7,310.9
 L112.2,310.6 L112.8,310.3 L113.3,310.1 L113.9,309.8 L114.4,309.5 L115.0,309.3 L115.5,309.0 L116.1,308.7
 L116.6,308.5 L117.2,308.2 L117.7,308.0 L118.3,307.7 L118.8,307.4 L119.4,307.2 L119.9,306.9 L120.5,306.7
 L121.0,306.4 L121.6,306.2 L122.1,305.9 L122.7,305.7 L123.2,305.4 L123.8,305.2 L124.3,304.9 L124.9,304.7
 L125.4,304.4 L126.0,304.2 L126.5,303.9 L127.1,303.7 L127.6,303.5 L128.2,303.2 L128.7,303.0 L129.3,302.7
 L129.8,302.5 L130.4,302.3 L130.9,302.0 L131.5,301.8 L132.0,301.6 L132.6,301.3 L133.1,301.1 L133.7,300.9
 L134.2,300.6 L134.8,300.4 L135.3,300.2 L135.9,299.9 L136.4,299.7 L137.0,299.5 L137.5,299.3 L138.1,299.0
 L138.6,298.8 L139.2,298.6 L139.7,298.4 L140.3,298.1 L140.8,297.9 L141.4,297.7 L141.9,297.5 L142.5,297.2
 L143.0,297.0 L143.6,296.8 L144.1,296.6 L144.7,296.3 L145.2,296.1 L145.8,295.9 L146.3,295.7 L146.9,295.5
 L147.4,295.2 L148.0,295.0 L148.5,294.8 L149.1,294.6 L149.6,294.4 L150.2,294.2 L150.7,293.9 L151.3,293.7
 L151.8,293.5 L152.4,293.3 L152.9,293.1 L153.5,292.9 L154.0,292.7 L154.6,292.4 L155.1,292.2 L155.7,292.0
 L156.2,291.8 L156.8,291.6 L157.3,291.4 L157.9,291.2 L158.4,291.0 L159.0,290.8 L159.5,290.5 L160.1,290.3
 L160.6,290.1 L161.2,289.9 L161.7,289.7 L162.3,289.5 L162.8,289.3 L163.4,289.1 L163.9,288.9 L164.5,288.7
 L165.0,288.4 L165.6,288.2 L166.1,288.0 L166.7,287.8 L167.2,287.6 L167.8,287.4 L168.3,287.2 L168.9,287.0
 L169.4,286.8 L170.0,286.6 L170.5,286.4 L171.1,286.2 L171.6,286.0 L172.2,285.7 L172.7,285.5 L173.3,285.3
 L173.8,285.1 L174.4,284.9 L174.9,284.7 L175.5,284.5 L176.0,284.3 L176.6,284.1 L177.1,283.9 L177.7,283.7
 L178.2,283.5 L178.8,283.3 L179.3,283.1 L179.9,282.8 L180.4,282.6 L181.0,282.4 L181.5,282.2 L182.1,282.0
 L182.6,281.8 L183.2,281.6 L183.7,281.4 L184.3,281.2 L184.8,281.0 L185.4,280.8 L185.9,280.6 L186.5,280.4
 L187.0,280.2 L187.6,279.9 L188.1,279.7 L188.7,279.5 L189.2,279.3 L189.8,279.1 L190.3,278.9 L190.9,278.7
 L191.4,278.5 L192.0,278.3 L192.5,278.1 L193.1,277.9 L193.6,277.7 L194.2,277.4 L194.7,277.2 L195.3,277.0
 L195.8,276.8 L196.4,276.6 L196.9,276.4 L197.5,276.2 L198.0,276.0 L198.6,275.8 L199.1,275.5 L199.7,275.3
 L200.2,275.1 L200.8,274.9 L201.3,274.7 L201.9,274.5 L202.4,274.3 L203.0,274.1 L203.5,273.8 

Points of separation are obtained by finding the positions where the value of the gradient of velocityX normal to the wall changes its sign. 

In [ ]:
ReattachmentPositionsTOP

key,value
"(1.9423, 200)",[ ]
"(1.9423, 250)","[ 9.06526517554962, 12.380523401538557 ]"
"(1.9423, 300)","[ 9.433618535858649, 15.346995943746165 ]"
"(1.9423, 350)","[ 9.88836809689491, 17.939189691013272 ]"
"(1.9423, 400)","[ 10.342316017472132, 20.288918358319005 ]"


In [ ]:
List<(double,double)> primaryReatLengths = new List<(double,double)>();
List<double> Rey = new List<double>();
List<double> Lenghths = new List<double>();

//Try to filter values in order to obtain the length of reattachment after the aparition of the primary vortex  
foreach(var a in ReattachmentPositions){
        double Reynolds = a.Key.Item2;
        double[] allLengths = a.Value;
        try{
            double primLength = allLengths[0];
            if(primLength < 0.3)
                primLength = allLengths[1];
Rey.Add(Reynolds);
Lenghths.Add(primLength);
            primaryReatLengths.Add((Reynolds,primLength));
        }catch(Exception e){
        continue;
        }
}

double[] Re_arr = Rey.ToArray();
double[] LeArr = Lenghths.ToArray();

Comparison of reattachment lengths for the primary vortex for ER =1.9423 

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;

fmt.LineColor = myCollors[0];
plot.AddDataGroup("BoSSS", Re_arr, LeArr, fmt);

// Reference data
MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];

string path2 = Path.Combine(Directory.GetCurrentDirectory(), @"reatlengths_bot.txt"); // CF_RE400_ER1data for ER 1.94...
ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
var fmt4 = new PlotFormat();
fmt4.Style = Styles.Points;

fmt4.PointType = myPointTypes[0];// PointTypes.Diamond;
fmt4.PointSize = 0.5;
var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
for(int i = 0; i < xArr_nu.Length; i++){
    xArr_nu[i]*=0.5; // REescale Reynolds for comparison with paper 
}

var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
plot.AddDataGroup("reference",xArr_nu,yArr_nu,fmt4);
plot . PlotNow ()


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 -50 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 350 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSS 
 
 
 BoSSS 
 
 
 
 
 
 reference 
 
 
 reference

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;

fmt.LineColor = myCollors[0];
plot.AddDataGroup("BoSSS_Deatach", Re_arr, LeArr, fmt);

// Reference data
MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];
string path2 = Path.Combine(Directory.GetCurrentDirectory(), @"reatlengths_bot.txt"); // CF_RE400_ER1data for ER 1.94...
ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
var fmt4 = new PlotFormat();
fmt4.Style = Styles.Points;

fmt4.PointType = myPointTypes[0];// PointTypes.Diamond;
fmt4.PointSize = 0.5;
var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
for(int i = 0; i < xArr_nu.Length; i++){
    xArr_nu[i]*=0.5; // REescale Reynolds for comparison with paper 
}

var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
plot.AddDataGroup("reference",xArr_nu,yArr_nu,fmt4);
plot . PlotNow ()


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 -50 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 350 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSSDeatach 
 
 
 BoSSS D eatach 
 
 
 
 
 
 reference 
 
 
 reference